In [1]:
#------------------ Script to Save Model Results to Postgre ------------------#

'''

-----------
INFORMATION
-----------

Author: Jean-Sebastien Roussy
Date Created: 2022-02-18
Last Modified: 2022-02-18

Description:

    Original App.py file was too large for Heroku Hobby Tier Dyno, so the 
    program was split in two to and model results table sent to 
    Heroku Postgre DB.
    
Modification History:

               
'''

'\n\n-----------\nINFORMATION\n-----------\n\nAuthor: Jean-Sebastien Roussy\nDate Created: 2022-02-18\nLast Modified: 2022-02-18\n\nDescription:\n\n    Original App.py file was too large for Heroku Hobby Tier Dyno, so the \n    program was split in two to and model results table sent to \n    Heroku Postgre DB.\n    \nModification History:\n\n               \n'

In [2]:
###############
### Imports ###
###############

from os import getenv
from os.path import join
from io import BytesIO, StringIO
from joblib import dump, load
import boto3
import tempfile
import csv
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import math
from math import pi

from sqlalchemy import Table, Column, Integer, Float, DateTime, Text, ForeignKey
from sqlalchemy import create_engine, MetaData

In [18]:
########################
### Global Variables ###
########################

dtype_pandasToSQL = {'int32': Integer,
                     'int64': Integer,
                     'float32': Float,
                     'float64': Float,
                     'object': Text,
                     'datetime64[ns, UTC]': DateTime}

In [21]:
#################
### Functions ###
#################

#################################
### Feature Engineering Funcs ###
#################################

# For time based columns
def transformation(column):
    max_value = column.max()
    sin_values = [math.sin((2*pi*x)/max_value) for x in list(column)]
    cos_values = [math.cos((2*pi*x)/max_value) for x in list(column)]
    return sin_values, cos_values

######################
### Read SQL to DF ###
######################

# Source: https://towardsdatascience.com/optimizing-pandas-read-sql-for-postgres-f31cd7f707ab
def read_sql_tmpfile(query, dbengine, table=None):
    with tempfile.TemporaryFile() as tmpfile:
        if table == 'matview':
            copy_sql = 'COPY (SELECT * FROM "{query}") TO STDOUT WITH CSV {head}'.format(
                        query=query, head="HEADER")
        else:
            copy_sql = 'COPY "{query}" TO STDOUT WITH CSV {head}'.format(
                        query=query, head="HEADER")
        conn = dbengine.raw_connection()
        cur = conn.cursor()
        cur.copy_expert(copy_sql, tmpfile)
        tmpfile.seek(0)
        df = pd.read_csv(tmpfile)
        return df
    
#######################
### S3 Loading Func ###
#######################

# Source: https://stackoverflow.com/questions/62941174/how-to-write-load-machine-learning-model-to-from-s3-bucket-through-joblib
def read_joblib(path):
    ''' 
       Function to load a joblib file from an s3 bucket or local directory.
       Arguments:
       * path: an s3 bucket or local directory path where the file is stored
       Outputs:
       * file: Joblib file loaded
    '''

    # Path is an s3 bucket
    if path[:5] == 's3://':
        s3_bucket, s3_key = path.split('/')[2], path.split('/')[3:]
        s3_key = '/'.join(s3_key)
        with BytesIO() as f:
            boto3.client("s3").download_fileobj(Bucket=s3_bucket, Key=s3_key, Fileobj=f)
            f.seek(0)
            file = load(f)
    
    return file

#####################
### Postgre Funcs ###
#####################

# List column names and dtypes from pandas dataframe
def pandas_data_info(df):
    col_lst = df.columns.tolist()
    dtype_lst = df.dtypes
    df_info_lst = list(zip(col_lst, dtype_lst))
    return df_info_lst

# Drop table if it exists with SQLALchemy
# Source: https://stackoverflow.com/questions/35918605/how-to-delete-a-table-in-sqlalchemy/66376565#66376565
def drop_table(table_name, dbengine):
    metadata = MetaData()
    metadata.reflect(bind=dbengine)
    with dbengine.connect() as conn:
        if table_name in metadata.tables:
            table = metadata.tables[table_name]
            print(f'Table {table_name} to be removed.')
            metadata.drop_all(conn, [table], checkfirst=True)
            print(f'Table {table_name} deleted!')
        else:
            print(f'Table {table_name} does not exist.')
            
# Create single table with SQLAlchemy
def create_table(dbengine, table_name, column_info):
    metadata = MetaData()
    with dbengine.connect() as conn:
        # Set new table and create it
        table = Table(name,
                      metadata,
                      #Column('id', Integer, primary_key=True),
                      *(Column(name, dtype) for name, dtype in column_info if name != 'id'))
        print(f'Table {name} created!')
        # Create all tables
        metadata.create_all(conn, checkfirst=False)
        return table

# Insert table values
# Alternative to_sql() *method* for DBs that support COPY FROM
# https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#io-sql-method
# https://stackoverflow.com/questions/23103962/how-to-write-dataframe-to-postgres-table
def psql_insert_copy(table, engine, keys, data_iter):
    """
    Execute SQL statement inserting data

    Parameters
    ----------
    table : pandas.io.sql.SQLTable
    conn  : sqlalchemy.engine.Engine or sqlalchemy.engine.Connection
    keys  : list of str
            Column names
    data_iter : Iterable that iterates the values to be inserted
    """
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = engine.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join(['"{}"'.format(k) for k in keys])
        if table.schema:
            table_name = '{}."{}"'.format(table.name) # Case sensitive table names require double quotes
        else:
            table_name = table.name

        sql = 'COPY "{}" ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)
        print(f'Data succesfully copied into {table_name}')

In [5]:
#######################
### POSTGRESQL INFO ###
#######################

# Load in environment file
load_dotenv()

# SQLAlchemy engine from env file
db_uri = getenv('SQLALCHEMY_NFL_URI')
if db_uri.startswith("postgres://"):
    db_uri = db_uri.replace("postgres://", "postgresql+psycopg2://", 1)

engine = create_engine(db_uri, 
                       convert_unicode=True, 
                       encoding='utf-8')

In [6]:
#################
### LOAD DATA ###
#################

# Load Statistics
team_stats_records_df = read_sql_tmpfile('team_stats_records', engine, table='matview')
# Load matches
matches_df = read_sql_tmpfile('matches', engine)

In [7]:
#########################
### JOIN DATA & CLEAN ###
#########################

# Merge Remerged df to Matches df
nfl_df = matches_df.merge(team_stats_records_df, left_on=['season', 'home_team'], 
                          right_on=['season','id'])\
                   .merge(team_stats_records_df, left_on=['season', 'away_team'], 
                          right_on=['season','id'],
                          suffixes=('_home', '_away'))

# Sort NFL df by date_time
nfl_df = nfl_df.sort_values(['date_time'])

# Drop redundant columns like id_x/id_y, etc
drop_cols = [col for col in nfl_df.columns if '_x' in col or '_y' in col]
nfl_df = nfl_df.drop(drop_cols, axis=1)

# Drop stats columns with 999 values in it
drop_999_cols = [col for col in nfl_df.columns[nfl_df.isin([999]).any()]\
                   if 'stat' in col]
nfl_df = nfl_df.drop(drop_999_cols, axis=1)

# Drop stats and record columns with only 0 values
drop_0_cols = [col for col, is_zero in ((nfl_df == 0).sum() == nfl_df.shape[0])\
               .items() if is_zero and ('stat' in col or 'record' in col)]
nfl_df = nfl_df.drop(drop_0_cols, axis=1)

# Replace 999 values in matches info
replace_999_cols = ['home_team_win', 'home_team_score', 
                    'away_team_win', 'away_team_score']
nfl_df[replace_999_cols] = nfl_df[replace_999_cols].replace(999,0)

# Replace remaining NaNs with 0
nfl_df = nfl_df.fillna(0)

In [8]:
###########################
### FEATURE ENGINEERING ###
###########################

# Date_time to Month, Day of Week, Time of Day as ints
nfl_df['date_time'] = pd.to_datetime(nfl_df['date_time'])
nfl_df['month'] = nfl_df['date_time'].dt.month.astype('int64')
nfl_df['day_week'] = nfl_df['date_time'].dt.dayofweek.astype('int64')
nfl_df['time_day'] = nfl_df['date_time'].dt.hour.astype('int64')

# Calculate Cosine and Sine for new dates
for col in ['month', 'day_week', 'time_day']:
    time_sine, time_cos = transformation(nfl_df[col])
    nfl_df[col[0:3]+'_sine'] = time_sine
    nfl_df[col[0:3]+'_cos'] = time_cos

In [9]:
###################
### X & y SPLIT ###
###################

# X cols list
keep_cols = ['stat', 'record', 'sine', 'cos',
             'division', 'conference', 'team_name', 'location']

remove_cols = ['team_win', 'team_score', 'id', 'date_time',
               'month', 'day_week', 'time_day']

X_cols =  [col for col in list(nfl_df.columns)\
                    if any(kc in col for kc in keep_cols)\
           and any(rc not in col for rc in remove_cols)]

## Split X and y into Train and Test
# X,y Train
X_train = nfl_df[~nfl_df['season'].isin([2021])][X_cols]
y_train = nfl_df[~nfl_df['season'].isin([2021])]['home_team_win']
y_regtrain = nfl_df[~nfl_df['season'].isin([2021])][['home_team_score',
                                                     'away_team_score']]
# X
X_test = nfl_df[nfl_df['season'].isin([2021])][X_cols]

In [10]:
##################
### LOAD MODEL ###
##################

# Load Logistic Regresssion Model
xgb_model = read_joblib(getenv('XGB_URI'))
# Load SVR Model
svr_model = read_joblib(getenv('SVR_URI'))

In [11]:
########################
### 2021 PREDICTIONS ###
########################

## 2021 DATA 

# Fit Model Logistic Regression
xgb_model.fit(X_train, y_train)

# Predict labels
pred_label = xgb_model.predict(X_test)

# Probabilities
prob = xgb_model.predict_proba(X_test)

# Fit Model SVR
svr_model.fit(X_train, y_regtrain)

# Predict Scores
pred_scores = svr_model.predict(X_test)

C:\Users\Jean\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:53:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




C:\Users\Jean\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\Users\Jean\anaconda3\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [12]:
###################
### 2021 TABLES ###
###################

# Weekly results Table
nfl_df2021 = nfl_df[nfl_df['season'] == 2021][['team_name_home',
                                               'home_team_score',
                                               'away_team_score',
                                               'team_name_away',
                                               'week',
                                               'home_team_win']]

# Change column names
nfl_df2021 = nfl_df2021.rename(columns={'team_name_home':'home_team',
                                        'team_name_away':'away_team',
                                        'home_team_score':'home_score',
                                        'away_team_score':'away_score'})

# Add probability column to df
nfl_df2021['home_team_win_probability'] = [int(round(x*100)) for x in prob[:,1]]
# Add predicted labels
nfl_df2021['predicted_winner'] = pred_label
# Add predicted Home Score
nfl_df2021['predicted_home_score'] = [int(round(x)) for x in pred_scores[:,0]]
# Add predicted Home Score
nfl_df2021['predicted_away_score'] = [int(round(x)) for x in pred_scores[:,1]]

# Map actual winner name to column
nfl_df2021['actual_winner'] = nfl_df2021.apply(lambda x: x.home_team if x.home_team_win == 1 else x.away_team, axis=1)
# Map predicted winner name to column
nfl_df2021['predicted_winner'] = nfl_df2021.apply(lambda x: x.home_team if x.predicted_winner == 1 else x.away_team, axis=1)
# Map win probability
nfl_df2021['win_probability'] = nfl_df2021.apply(lambda x: x.home_team_win_probability if x.predicted_winner == x.home_team else 100 - x.home_team_win_probability, axis=1)
# Map correct predicition
nfl_df2021['correct_prediction'] = (nfl_df2021['predicted_winner'] == nfl_df2021['actual_winner']).astype(int)
# Replace Actual Winner value with None if game not played yet
nfl_df2021['actual_winner'] = np.where((nfl_df2021['home_score'] == 0) & (nfl_df2021['away_score'] == 0), "None", nfl_df2021['actual_winner'])
# Replace Correct Prediction value with -1 if game not played yet
nfl_df2021['correct_prediction'] = np.where((nfl_df2021['home_score'] == 0) & (nfl_df2021['away_score'] == 0), -1, nfl_df2021['correct_prediction'])

# Drop columns
nfl_df2021 = nfl_df2021.drop(columns=['home_team_win_probability', 'home_team_win'])

# Move Cols
move_cols = nfl_df2021.columns.to_list()
new_order = move_cols[4:5] + move_cols[0:2] + move_cols[6:8] + move_cols[2:4] + move_cols[5:6] + move_cols[8:]
nfl_df2021 = nfl_df2021[new_order]

In [19]:
#################
### DATA INFO ###
#################

# Grab column names
# Add DF col/dtype info to var
col_info = pandas_data_info(nfl_df2021)
    
# Convert dtype to match PostgreSQL dtypes and add to empty list
ncol_type_lst = []
for col, dtype in col_info:
    if dtype_pandasToSQL[str(dtype)]:
        dtype = dtype_pandasToSQL[str(dtype)]
        ncol_type_lst.append((col, dtype))

[('week', <class 'sqlalchemy.sql.sqltypes.Integer'>), ('home_team', <class 'sqlalchemy.sql.sqltypes.Text'>), ('home_score', <class 'sqlalchemy.sql.sqltypes.Integer'>), ('predicted_home_score', <class 'sqlalchemy.sql.sqltypes.Integer'>), ('predicted_away_score', <class 'sqlalchemy.sql.sqltypes.Integer'>), ('away_score', <class 'sqlalchemy.sql.sqltypes.Integer'>), ('away_team', <class 'sqlalchemy.sql.sqltypes.Text'>), ('predicted_winner', <class 'sqlalchemy.sql.sqltypes.Text'>), ('actual_winner', <class 'sqlalchemy.sql.sqltypes.Text'>), ('win_probability', <class 'sqlalchemy.sql.sqltypes.Integer'>), ('correct_prediction', <class 'sqlalchemy.sql.sqltypes.Integer'>)]


In [22]:
#####################
### CREATE TABLES ###
#####################

# NFL DF table name
name = 'nfl_2021_season'

# Delete table if exists
drop_table(name, engine)

# Create table
nfl_table = create_table(engine, name, ncol_type_lst)

Table nfl_2021_season to be removed.
Table nfl_2021_season deleted!
Table nfl_2021_season created!


In [23]:
###################
### INSERT DATA ###
###################

# Add teams table data first
nfl_df2021.to_sql(name, engine, if_exists='append', index=False, method=psql_insert_copy)
    

Data succesfully copied into nfl_2021_season
